In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from pydataset import data

Paper by: Hadley Wickham

https://vita.had.co.nz/papers/tidy-data.pdf

### Data cleaning:

tidy data != clean data


- outlier checking  
- date parsing
- missing value imputation etc.
- <ins>data tidying: structuring datasets to facilitate analysis.</ins>



### Data semantics: tidy data

- Value: every value belongs to a variable and an observation.
- Variable: a variable contains all values that measure the same underlying attribute (like height, temperature, duration) across units.
- Observation: an observation contains all values measured on the same unit (like a person, or a day, or a race) across attributes.

####  datasets for lesson and exercises:

- https://classroom.google.com/c/MzUxODYzODI3NTE5/m/MzUxODYzODI3Njg5/details

In [2]:
# lets look at this data:
treatments = pd.read_csv('untidy-data/treatment.csv')
treatments

,Unnamed: 0,treatmenta,treatmentb,treatmentc
0,John Smith,NaN,2,0
1,Jane Doe,16.0,11,3
2,Mary Johnson,3.0,1,4


In [3]:
#rename columns
treatments.columns = ['name', 'a', 'b', 'c']
treatments

,name,a,b,c
0,John Smith,NaN,2,0
1,Jane Doe,16.0,11,3
2,Mary Johnson,3.0,1,4


What is an observation here?  
variables?  
values?  

In [4]:
# restructure data using 'melt'

treatments = treatments.melt(id_vars=['name'], var_name='treatment', value_name='response')

In [5]:
treatments

,name,treatment,response
0,John Smith,a,NaN
1,Jane Doe,a,16.0
2,Mary Johnson,a,3.0
3,John Smith,b,2.0
4,Jane Doe,b,11.0
5,Mary Johnson,b,1.0
6,John Smith,c,0.0
7,Jane Doe,c,3.0
8,Mary Johnson,c,4.0


### Tidy data : 
- Each variable forms a column.
- Each observation forms a row.
- Each cell has a single value.
- data is tabular, i.e. made up of rows and columns

In [ ]:
# Examples of tidy-data?



#### General Ideas  
- If the units are the same, maybe they should be in the same column.
- If one column has measurements of different units, it should be spread out  
- Should you be able to groupby some of the columns? combine them  
- Can I pass this data to seaborn?  

- Can we ask interesting questions and answer them with a groupby? i.e. generally we don't want to be taking row or column averages.


### How to deal with 'messy' data


##### Reshaping data:  
  
- Wide data --> Long data format (Melt)  
- Long data --> Wide Data format (pivot_table, unstack)  

#### 1. Messy data: Column headers are values, not variable names.

In [6]:
df = pd.read_csv('untidy-data/pew.csv')
df.head()

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/refused
0,Agnostic,27,34,60,81,76,137,122,109,84,96
1,Atheist,12,27,37,52,35,70,73,59,74,76
2,Buddhist,27,21,30,34,33,58,62,39,53,54
3,Catholic,418,617,732,670,638,1116,949,792,633,1489
4,Don’t know/refused,15,14,15,11,10,35,21,17,18,116


In [7]:
# look at info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   religion            18 non-null     object
 1   <$10k               18 non-null     int64 
 2   $10-20k             18 non-null     int64 
 3   $20-30k             18 non-null     int64 
 4   $30-40k             18 non-null     int64 
 5   $40-50k             18 non-null     int64 
 6   $50-75k             18 non-null     int64 
 7   $75-100k            18 non-null     int64 
 8   $100-150k           18 non-null     int64 
 9   >150k               18 non-null     int64 
 10  Don't know/refused  18 non-null     int64 
dtypes: int64(10), object(1)
memory usage: 1.7+ KB


In [9]:
# Melt the data. Creates two new columns with deault name of 'variable' and 'value'

df.melt(id_vars = 'religion')


,religion,variable,value
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don’t know/refused,<$10k,15
...,...,...,...
175,Orthodox,Don't know/refused,73
176,Other Christian,Don't know/refused,18
177,Other Faiths,Don't know/refused,71
178,Other World Religions,Don't know/refused,8


In [11]:
# melt data and specify names of new columns

df_tidy = df.melt(id_vars = 'religion', 
                  var_name = 'income', 
                  value_name = 'count')
df_tidy

,religion,income,count
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don’t know/refused,<$10k,15
...,...,...,...
175,Orthodox,Don't know/refused,73
176,Other Christian,Don't know/refused,18
177,Other Faiths,Don't know/refused,71
178,Other World Religions,Don't know/refused,8


#### pd.melt arguments
- id_vars = columns you want to keep (not melt)
- var_name = name of new column you created by melting columns
- value_name = column name for values

#### Another example: one variable stored across multiple columns

In [12]:
billboard = pd.read_csv('untidy-data/billboard.csv', encoding= 'unicode_escape')

In [13]:
billboard.head()

,year,artist,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,9/23/00,11/18/00,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2/12/00,4/8/00,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,10/23/99,1/29/00,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,8/12/00,9/16/00,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,8/5/00,10/14/00,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# what is mean rating for each track?


In [ ]:
# melt the data

billboard_melt

In [ ]:
# what is the mean rating (across all weeks) of each track?
billboard_melt.groupby('track').rating.mean()

#### 2. Messy data: Multiple variables are stored in one column.

In [15]:
df = pd.DataFrame({
    'name': ['Sally', 'Jane', 'Billy', 'Suzy'],
    'pet': ['dog: max', 'dog: buddy', 'cat: grizabella', 'hamster: fred']
})
df

,name,pet
0,Sally,dog: max
1,Jane,dog: buddy
2,Billy,cat: grizabella
3,Suzy,hamster: fred


In [ ]:
# how to split the string


In [ ]:
# how to split a pandas column/series


In [ ]:
# Another (better) way


In [ ]:
# create two new columns


In [ ]:
df.head()

#### Messy data: Variables are stored in both rows and columns

In [17]:
weather = pd.read_csv('untidy-data/weather.csv')

weather.head()

,id,year,month,element,d1,d2,d3,d4,d5,d6,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.8,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.5,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,...,NaN,29.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,...,NaN,10.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# how many rows and columns?
weather.shape

(22, 35)

In [19]:
# melt the 'days'

weather_long = weather.melt(id_vars = ['id', 'year', 'month', 'element'],
                           var_name = 'day',
                           value_name = 'temp')

weather_long.head()

,id,year,month,element,day,temp
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN
4,MX17004,2010,3,tmax,d1,NaN


In [ ]:
weather_long.shape

In [ ]:
# pivot the element column. Reset index to go from multi-index to flat dataframe.

# weather_tidy = 

#### pd.pivot_table arguments
- Index = columns you want to keep (not pivot)
- columns = column you want to pivot
- values = values we want to populate in the new columns
- aggfunct = how you want to aggregate the duplicate rows

## Mini Exercise:

- read in excel file named dem_score.xls
- convert the data in tidy format (hint: melt the data)
- convert the melted dataframe back in wide-format (hint: pivot the data)




In [20]:
# read in the excel file named dem_score.xlsx
df = pd.read_excel('untidy-data/dem_score.xlsx')
df.head()

,country,1952,1957,1962,1967,1972,1977,1982,1987,1992
0,Albania,-9,-9,-9,-9,-9,-9,-9,-9,5
1,Argentina,-9,-1,-1,-9,-9,-9,-8,8,7
2,Armenia,-9,-7,-7,-7,-7,-7,-7,-7,7
3,Australia,10,10,10,10,10,10,10,10,10
4,Austria,10,10,10,10,10,10,10,10,10


In [21]:
# is the data in tidy format? What is the shape of data

df.shape

(96, 10)

In [29]:
# convert dataframe in 'tidy' format (hint: pd.melt)

df_long = pd.melt(df, id_vars = 'country',
                   var_name = 'year',
                   value_name = 'score')

df_long.head()

,country,year,score
0,Albania,1952,-9
1,Argentina,1952,-9
2,Armenia,1952,-9
3,Australia,1952,10
4,Austria,1952,10


In [25]:
# check shape of tidy dataframe
df_long.shape

(864, 3)

In [28]:
# convert the data back in wide format (hint: pivot_table)
df_long.pivot_table(index = 'country',
                   columns = 'year',
                   values = 'score').reset_index()


year,country,1952,1957,1962,1967,1972,1977,1982,1987,1992
0,Albania,-9,-9,-9,-9,-9,-9,-9,-9,5
1,Argentina,-9,-1,-1,-9,-9,-9,-8,8,7
2,Armenia,-9,-7,-7,-7,-7,-7,-7,-7,7
3,Australia,10,10,10,10,10,10,10,10,10
4,Austria,10,10,10,10,10,10,10,10,10
5,Azerbaijan,-9,-7,-7,-7,-7,-7,-7,-7,1
6,Belarus,-9,-7,-7,-7,-7,-7,-7,-7,7
7,Belgium,10,10,10,10,10,10,10,10,10
8,Bhutan,-10,-10,-10,-10,-10,-10,-10,-10,-10
9,Bolivia,-4,-3,-3,-4,-7,-7,8,9,9


#### Bit more complex example

In [30]:
sales = pd.read_csv('untidy-data/sales.csv')

In [31]:
#check head
sales.head()

,Product,2016 Sales,2016 PPU,2017 Sales,2017 PPU,2018 Sales,2018 PPU
0,A,673,5,231,7,173,9
1,B,259,3,748,5,186,8
2,C,644,3,863,5,632,5
3,D,508,9,356,11,347,14


In [32]:
#check the shape
sales.shape

(4, 7)

In [ ]:
# first we melt all the columns expect 'Product'

sales_melt[['year', 'measure']] = sales_melt.variable.str.split(' ', expand = True)

In [ ]:
# shape of melted dataset


In [ ]:
# here we split the 'variable' column to create two new columns 'year' and 'measure'
sales_melt[['year',]]

In [ ]:
# now we will pivot the 'measure' column to create the two new columns 'PPU' and 'Sales'
# reset the index to flatten the dataframe 

# sales_tidy 